In [1]:
import time, os, sys, subprocess, shutil
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [2]:
def sp_grep(file, match):
    o = subprocess.run('grep "{}" {}'.format(match, file), check=True, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    return o.stdout
def epoch_tail(grepopt):
    epochs = [i.split('||')[0].strip()[-2:].strip() for i in grepopt.split('\n') if i != '']
    tails = [i.split(',')[-1].split('.')[0] for i in grepopt.split('\n') if i != '']
    return (epochs, tails)


In [3]:
class FileCopier():
    def __init__(self, prefix):
        self.prefix = prefix

class CheckpointHandler(FileSystemEventHandler):
    def __init__(self, watchDir='', targetDir = ''):
        self.watchDir = watchDir
        self.targetDir = targetDir
        self.prefix = None
        self.log = None
        
    def on_created(self, event):
        if 'log' in event.src_path:
            print("EVENT -- CREATED: {}".format(event.src_path))
            prefix = '.'.join(event.src_path.split('/')[-1].split('.')[:2])
            self.prefix = prefix
            self.log = os.path.join(self.watchDir, prefix+'.log')
        if '.pt' in event.src_path:
            print("EVENT -- CREATED: {}".format(event.src_path))
            o = sp_grep(self.log, match='Epoch')
            es, ts = epoch_tail(grepopt = o)
            print(es, ts)
            for idx, e in enumerate(es):
                if len(e) == 1:
                    e = '0'+e
                p = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e))
                pt = os.path.join(self.watchDir, self.prefix+'_{}'.format(idx%3)+'.pt')
                tpt = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), self.prefix+'_{}'.format(idx%3)+'.pt')
                sl = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), 'xc')
                try:
                    print("os.mkdir({})".format(p))
                    os.mkdir(p)
                    print('waiting...')
                    time.sleep(10)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #if directory exists, continue to next file
                    continue
                try:
                    print('waiting...')
                    time.sleep(10)
                    print("shutil.copy:")
                    print("{} -> {}".format(pt, tpt))
                    shutil.copy(src=pt, dst=tpt)
                    print('os.symlink: {} -> {}'.format(tpt, sl))
                    os.symlink(tpt, sl)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #either directory exists or file does not yet exist
                    pass

        
    def on_modified(self, event):
        if 'log' in event.src_path and not self.prefix:
            print("EVENT -- MODIFIED: {}".format(event.src_path))
            prefix = '.'.join(event.src_path.split('/')[-1].split('.')[:2])
            self.prefix = prefix
            self.log = os.path.join(self.watchDir, prefix+'.log')
        if '.pt' in event.src_path:
            print("EVENT -- MODIFIED: {}".format(event.src_path))
            o = sp_grep(self.log, match='Epoch')
            es, ts = epoch_tail(grepopt = o)
            print(es, ts)
            for idx, e in enumerate(es):
                if len(e) == 1:
                    e = '0'+e
                p = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e))
                pt = os.path.join(self.watchDir, self.prefix+'_{}'.format(idx%3)+'.pt')
                tpt = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), self.prefix+'_{}'.format(idx%3)+'.pt')
                sl = os.path.join(self.targetDir, 'MODEL_MGGA_e{}'.format(e), 'xc')
                try:
                    print("os.mkdir({})".format(p))
                    os.mkdir(p)
                    print('waiting...')
                    time.sleep(10)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #if directory exists, continue to next file
                    continue
                try:
                    print('waiting...')
                    time.sleep(10)
                    print("shutil.copy:")
                    print("{} -> {}".format(pt, tpt))
                    shutil.copy(src=pt, dst=tpt)
                    print('os.symlink: {} -> {}'.format(tpt, sl))
                    os.symlink(tpt, sl)
                except:
                    e = sys.exc_info()[0]
                    print(e)
                    #either directory exists or file does not yet exist
                    pass 

In [4]:
obs = Observer()
watchdir = '/home/awills/Documents/Research/swxcd/train/test_subset2/pts3'
targdir = '/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3'
hand = CheckpointHandler(watchDir = watchdir, targetDir = targdir)
obs.schedule(hand, path=watchdir)
obs.start()

EVENT -- CREATED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750.log
EVENT -- CREATED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_current.pt
['0'] ['']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00)
waiting...
waiting...
shutil.copy:
/home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_0.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00/2022-08-10_16:48:21.436750_0.pt
os.symlink: /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00/2022-08-10_16:48:21.436750_0.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00/xc
EVENT -- MODIFIED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_current.pt
['0'] ['']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00)
<class 'FileExis

waiting...
shutil.copy:
/home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_1.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e04/2022-08-10_16:48:21.436750_1.pt
os.symlink: /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e04/2022-08-10_16:48:21.436750_1.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e04/xc
EVENT -- MODIFIED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_current.pt
['0', '1', '2', '3', '4'] ['', '', '', '', '']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e01)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e02)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pt

waiting...
shutil.copy:
/home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_1.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e07/2022-08-10_16:48:21.436750_1.pt
os.symlink: /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e07/2022-08-10_16:48:21.436750_1.pt -> /home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e07/xc
EVENT -- MODIFIED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_current.pt
['0', '1', '2', '3', '4', '5', '6', '7'] ['', '', '', '', '', '', '', '']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e01)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e02)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research

EVENT -- MODIFIED: /home/awills/Documents/Research/swxcd/train/test_subset2/pts3/2022-08-10_16:48:21.436750_current.pt
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] ['', '', '', '', '', '', '', '', '', '', '']
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e00)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e01)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e02)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e03)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e04)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e05)
<class 'FileExistsError'>
os.mkdir(/home/awills/Documents/Research/swxcd/eval/test_subset2/pts3/MODEL_MGGA_e06)
<class 'FileExistsErr

In [5]:
obs.stop()
obs.join()

In [14]:
tsp = '/home/awills/Documents/Research/swxcd/train/test_subset/ptscan/2022-08-01_17:18:57.321534.log'
evaldir = '/home/awills/Documents/Research/swxcd/eval/test_subset/pts1'
if 'log' in tsp:
    prefix = '.'.join(tsp.split('/')[-1].split('.')[:2])
    
    

In [15]:
prefix

'2022-08-01_17:18:57.321534'

In [54]:
o.split('\n')

["Epoch 0 || ,[' rho : 2.883240', ' ae : 52.820392', ' E : 1.169998'] ,|| total loss 39.512935 ,_0.chkpt",
 "Epoch 1 || ,[' rho : 2.887368', ' ae : 52.629850', ' E : 1.334795'] ,|| total loss 39.391703 ,_1.chkpt",
 '']

In [51]:
[i.split(',')[-1].split('.')[0] for i in o.split('\n') if i != '']

['_0', '_1']

In [52]:
[i.split('||')[0].strip()[-2:].strip() for i in o.split('\n') if i != '']

['0', '1']

In [17]:
prefix

'2022-08-01_17:18:57321534'

In [ ]:
try:
    while True:
        time.sleep(100)
except:
    obs.stop()
    
obs.join()